In [66]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

In [82]:


import os
path = "C:/Users/bruno/OneDrive/Área de Trabalho/Teste/soja" # replace with your folder path
data_dir = pathlib.Path(path)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

#define some parameters to the loader
batch_size = 32
img_height = 227
img_width = 227

5513


In [93]:
#Creating a list of images
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)
#Creating the class list
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: C:\\Users\\bruno\\OneDrive\\\xc3\x81rea de Trabalho\\Teste\\soja\\*\\*'

In [78]:
#splitting the dataset into train, test and validation sets
train_size = int(0.7 * image_count)
test_size = int(0.15 * image_count)

train_ds = list_ds.take(train_size)
test_ds = list_ds.skip(train_size)
val_ds = test_ds.skip(test_size)
test_ds = test_ds.take(test_size)

print('Training size:', len(train_ds))
print('Validation size:', len(val_ds))
print('Test size:', len(test_ds))

Training size: 21
Validation size: 0
Test size: 0


In [57]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  one_hot = parts[-2] == class_names
  return tf.argmax(one_hot)

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label


In [58]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))

In [59]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [60]:
def inception_module(x):
  # branch with 1x1 conv
  branch1x1 = layers.Conv2D(64, 1, activation='relu')(x)
  branch3x3 = layers.Conv2D(64, 1, activation='relu')(x)
  branch3x3 = layers.Conv2D(96, 3, padding='same', activation='relu')(branch3x3)
  branch5x5 = layers.Conv2D(64, 1, activation='relu')(x)
  branch5x5 = layers.Conv2D(96, 5, padding='same', activation='relu')(branch5x5)  
  branchPol = layers.MaxPooling2D(3, strides=1, padding='same')(x)
  branchPol = layers.Conv2D(96, 1, activation='relu')(branchPol)
  output = layers.concatenate([branch1x1, branch3x3, branch5x5, branchPol], axis=3)
  return output

In [61]:

input_img = Input(shape=(227, 227, 3))  # Example input shape (224x224 RGB image)

# Apply initial convolution and max pooling
x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_img)
x = layers.MaxPooling2D((2, 2))(x)

# Apply the Inception module
x = inception_module(x)

# Apply additional convolution and pooling layers
x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Flatten and add dense layers
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Add dropout for regularization
x = layers.Dense(len(class_names), activation='softmax')(x)  # Output layer with 10 classes (example)

# Create the model
model = Model(inputs=input_img, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Print the model summary
model.summary()

In [64]:
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 158s 100ms/step - accuracy: 0.3559 - loss: 1.8084 - val_accuracy: 0.5345 - val_loss: 1.3212
Epoch 2/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 168s 108ms/step - accuracy: 0.5685 - loss: 1.2344 - val_accuracy: 0.5915 - val_loss: 1.1566
Epoch 3/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 158s 101ms/step - accuracy: 0.6548 - loss: 1.0032 - val_accuracy: 0.6254 - val_loss: 1.0781
Epoch 4/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 160s 102ms/step - accuracy: 0.7126 - loss: 0.8318 - val_accuracy: 0.6156 - val_loss: 1.1361
Epoch 5/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 154s 99ms/step - accuracy: 0.7626 - loss: 0.6871 - val_accuracy: 0.6306 - val_loss: 1.1814
Epoch 6/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 155s 99ms/step - accuracy: 0.8197 - loss: 0.5240 - val_accuracy: 0.6343 - val_loss: 1.2164
Epoch 7/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 155s 99ms/step - accuracy: 0.8658 - loss: 0.3916 - val_accuracy: 0.6289 - val_loss: 1.3353
Epoch 8/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 154s 98ms/step - accura

In [65]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

ValueError: The filename must end in `.weights.h5`. Received: filepath=c:\Users\bruno\OneDrive\Área de Trabalho\Teste\model.h5

In [ ]:
scores = model.evaluate(X_test, y_test, verbose = 0)

